## Install TensorTrade

## Setup Data Fetching

In [1]:
import ssl
import pandas as pd

from tensortrade.utils import CryptoDataDownload

ssl._create_default_https_context = ssl._create_unverified_context # Only used if pandas gives a SSLError

cdd = CryptoDataDownload()

In [2]:
data = pd.concat([
    cdd.fetch("Coinbase", "USD", "BTC", "1h").add_prefix("BTC:")
], axis=1)
data = data.rename({"BTC:date": "date"}, axis=1)

In [3]:
data.head()

,date,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2017-07-01 11:00:00,2505.56,2513.38,2495.12,2509.17,287000.32
1,2017-07-01 12:00:00,2509.17,2512.87,2484.99,2488.43,393142.50
2,2017-07-01 13:00:00,2488.43,2488.43,2454.40,2454.43,693254.01
3,2017-07-01 14:00:00,2454.43,2473.93,2450.83,2459.35,712864.80
4,2017-07-01 15:00:00,2459.35,2475.00,2450.00,2467.83,682105.41


## Create features with the data module

In [4]:
from tensortrade.data import Node, Module, DataFeed, Stream, Select


def rsi(price: Node, period: float):
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Node, fast: float, slow: float, signal: float) -> Node:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[1:]:
    s = Stream(list(data[c])).rename(data[c].name)
    features += [s]

btc_close = Select("BTC:close")(*features)

features += [
    rsi(btc_close, period=20).rename("BTC:rsi"),
    macd(btc_close, fast=10, slow=50, signal=5).rename("BTC:macd")
]
        
feed = DataFeed(features)
feed.compile()

In [5]:
feed.next()

/opt/conda/lib/python3.7/site-packages/tensortrade/data/stream/node.py:933: RuntimeWarning: invalid value encountered in double_scalars
  v = (w[::-1] * x).sum() / w.sum()


{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'BTC:rsi': nan,
 'BTC:macd': 0.0}

## Setup Trading Environment

In [6]:
from tensortrade.exchanges import Exchange
from tensortrade.exchanges.services.execution.simulated import execute_order
from tensortrade.data import Stream, DataFeed, Module
from tensortrade.instruments import USD, BTC, ETH
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.environments import TradingEnvironment


coinbase = Exchange("coinbase", service=execute_order)(
    Stream(list(data["BTC:close"])).rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(coinbase, 10_000 * USD),
    Wallet(coinbase, 2 * BTC)
])

env = TradingEnvironment(
    feed=feed,
    portfolio=portfolio,
    use_internal=False,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    window_size=20
)

## Example Data Feed Observation

Even though this observation contains data from the internal data feed, since `use_internal=False` this data will not be provided as input to the observation history. The data that will be added to observation history of the environment will strictly be the nodes that have been included into the data feed that has been provided as a parameter to the trading environment.

In [7]:
env.feed.next()

{'BTC:open': 2505.56,
 'BTC:high': 2513.38,
 'BTC:low': 2495.12,
 'BTC:close': 2509.17,
 'BTC:volume': 287000.32,
 'BTC:rsi': 0.0,
 'BTC:macd': -0.23222985476439617,
 'coinbase:/USD-BTC': 2509.17,
 'coinbase:/USD:/free': 10000.0,
 'coinbase:/USD:/locked': 0.0,
 'coinbase:/USD:/total': 10000.0,
 'coinbase:/BTC:/free': 2.0,
 'coinbase:/BTC:/locked': 0.0,
 'coinbase:/BTC:/total': 2.0,
 'coinbase:/BTC:/worth': 5018.34,
 'net_worth': 15018.34}

## Setup and Train DQN Agent

In [8]:
from tensortrade.agents import DQNAgent

agent = DQNAgent(env)

agent.train(n_steps=200, save_path="examples/agents/")

====      AGENT ID: 3ab3f2e6-debc-47b2-947a-5fdc65d9d506      ====
[2020-05-29 12:48:11 PM] Episode: 1/2147483647 Step: 150/200
[2020-05-29 12:48:14 PM] Episode: 1/2147483647 Step: 200/200
[2020-05-29 12:48:18 PM] Episode: 2/2147483647 Step: 50/200
[2020-05-29 12:48:21 PM] Episode: 2/2147483647 Step: 100/200
[2020-05-29 12:48:25 PM] Episode: 2/2147483647 Step: 150/200
[2020-05-29 12:48:28 PM] Episode: 2/2147483647 Step: 200/200
[2020-05-29 12:48:32 PM] Episode: 3/2147483647 Step: 50/200
[2020-05-29 12:48:35 PM] Episode: 3/2147483647 Step: 100/200
[2020-05-29 12:48:38 PM] Episode: 3/2147483647 Step: 150/200
[2020-05-29 12:48:42 PM] Episode: 3/2147483647 Step: 200/200
[2020-05-29 12:48:45 PM] Episode: 4/2147483647 Step: 50/200
[2020-05-29 12:48:49 PM] Episode: 4/2147483647 Step: 100/200
[2020-05-29 12:48:52 PM] Episode: 4/2147483647 Step: 150/200
[2020-05-29 12:48:56 PM] Episode: 4/2147483647 Step: 200/200
[2020-05-29 12:48:59 PM] Episode: 5/2147483647 Step: 50/200
[2020-05-29 12:49:02 P

KeyboardInterrupt: 

## Plot Performance

In [ ]:
%matplotlib inline

portfolio.performance.plot()

In [ ]:
portfolio.performance.net_worth.plot()

## Setup and Train Parallel DQN Agent

In [ ]:
from tensortrade.agents import ParallelDQNAgent

def create_env():
    
    env = TradingEnvironment(
        feed=feed,
        portfolio=portfolio,
        action_scheme='managed-risk',
        reward_scheme='risk-adjusted',
        window_size=20
    )
    
    return env

agent = ParallelDQNAgent(create_env)

agent.train(n_envs=4, n_steps=200, save_path="examples/agents/")